In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit

# Using pip
#!pip install torch torchvision torchaudio

# Or using conda
#!conda install pytorch torchvision torchaudio -c pytorch

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor



In [2]:
import pandas as pd

# Load the files and print their column names
wind_df = pd.read_csv('combined_wind_data.csv')
lmp_df = pd.read_csv('miso_lmp_hourly_data_minnesota.csv')
temp_df = pd.read_csv('temp_data_minnesota.csv')

print("Wind data columns:")
print(wind_df.columns.tolist())
print("\nTemperature data columns:")
print(temp_df.columns.tolist())
print("\nLMP data columns:")
print(lmp_df.columns.tolist())

Wind data columns:
['UTC Timestamp (Interval Ending)', 'Local Timestamp Eastern Standard Time (Interval Beginning)', 'Local Timestamp Eastern Standard Time (Interval Ending)', 'Local Date', 'Hour Number', 'MISO Total Wind Generation (MW)', 'source_file']

Temperature data columns:
['Minneapolis Temperature (Fahrenheit)', 'Minneapolis Temperature Observation Time (Eastern Standard)', 'UTC Timestamp (Interval Ending)', 'Local Timestamp Eastern Standard Time (Interval Beginning)', 'Local Timestamp Eastern Standard Time (Interval Ending)', 'Local Date', 'Hour Number', 'source_file']

LMP data columns:
['UTC Timestamp (Interval Ending)', 'Local Timestamp Eastern Standard Time (Interval Beginning)', 'Local Timestamp Eastern Standard Time (Interval Ending)', 'Local Date', 'Hour Number', 'Minnesota Hub LMP', 'Minnesota Hub (Congestion)', 'Minnesota Hub (Loss)', 'source_file']


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

class MISOPricePredictor:
    def __init__(self):
        self.scaler = StandardScaler()
        
    def load_and_prepare_data(self, generation_path, lmp_path, temp_path):
        """
        Load and prepare data from the three CSV files
        generation_path: hourly generation mix data
        lmp_path: hourly price data
        temp_path: hourly temperature data
        """
        # Load datasets
        print("Loading datasets...")
        gen_df = pd.read_csv(generation_path)
        lmp_df = pd.read_csv(lmp_path)
        temp_df = pd.read_csv(temp_path)
        
        # Convert timestamps
        print("Processing timestamps...")
        for df in [gen_df, lmp_df, temp_df]:
            df['timestamp'] = pd.to_datetime(
                df['UTC Timestamp (Interval Ending)'],
                format='mixed'
            )
            
        # Filter for 4 full years through December 2023
        end_date = '2023-12-31 23:00:00'
        start_date = '2020-01-01 00:00:00'
        
        print(f"Filtering data from {start_date} to {end_date}...")
        gen_df = gen_df[(gen_df['timestamp'] >= start_date) & (gen_df['timestamp'] <= end_date)]
        lmp_df = lmp_df[(lmp_df['timestamp'] >= start_date) & (lmp_df['timestamp'] <= end_date)]
        temp_df = temp_df[(temp_df['timestamp'] >= start_date) & (temp_df['timestamp'] <= end_date)]
            
        # Set index
        gen_df.set_index('timestamp', inplace=True)
        lmp_df.set_index('timestamp', inplace=True)
        temp_df.set_index('timestamp', inplace=True)
        
        # Sort indices
        gen_df.sort_index(inplace=True)
        lmp_df.sort_index(inplace=True)
        temp_df.sort_index(inplace=True)
        
        print("Extracting data...")
        # Extract generation columns
        generation_columns = [
            'MISO Total Total Generation (MW)',
            'MISO Total Coal Generation (MW)',
            'MISO Total Gas Generation (MW)',
            'MISO Total Hydro Generation (MW)',
            'MISO Total Nuclear Generation (MW)',
            'MISO Total Other Generation (MW)',
            'MISO Total Wind Generation (MW)',
            'MISO Total Solar Generation (MW)',
            'MISO Total Storage Generation (MW)'
        ]
        
        # Extract data and fill missing values with 0 for generation
        generation_data = gen_df[generation_columns].fillna(0)
        temp_data = temp_df['Minneapolis Temperature (Fahrenheit)']
        price_data = lmp_df[['Minnesota Hub LMP', 'Minnesota Hub (Congestion)', 'Minnesota Hub (Loss)']]
        
        # Create complete date range for the period
        date_range = pd.date_range(start=start_date, end=end_date, freq='h')
        
        # Reindex all dataframes to ensure complete timeline
        generation_data = generation_data.reindex(date_range).fillna(0)
        temp_data = temp_data.reindex(date_range)
        price_data = price_data.reindex(date_range)
        
        # Create initial DataFrame with only target variable (price)
        df = pd.DataFrame({'price': price_data['Minnesota Hub LMP']})
        
        # Store other variables for creating lagged features
        self.historical_data = {
            'congestion': price_data['Minnesota Hub (Congestion)'],
            'loss': price_data['Minnesota Hub (Loss)'],
            'temperature': temp_data,
            'total_gen': generation_data['MISO Total Total Generation (MW)'],
            'coal_gen': generation_data['MISO Total Coal Generation (MW)'],
            'gas_gen': generation_data['MISO Total Gas Generation (MW)'],
            'hydro_gen': generation_data['MISO Total Hydro Generation (MW)'],
            'nuclear_gen': generation_data['MISO Total Nuclear Generation (MW)'],
            'other_gen': generation_data['MISO Total Other Generation (MW)'],
            'wind_gen': generation_data['MISO Total Wind Generation (MW)'],
            'solar_gen': generation_data['MISO Total Solar Generation (MW)'],
            'storage_gen': generation_data['MISO Total Storage Generation (MW)']
        }
        
        # Calculate and store generation percentages for historical features
        for gen_type in ['coal', 'gas', 'hydro', 'nuclear', 'other', 'wind', 'solar', 'storage']:
            gen_value = self.historical_data[f'{gen_type}_gen']
            total_gen = self.historical_data['total_gen']
            self.historical_data[f'{gen_type}_pct'] = gen_value / (total_gen + 1e-10) * 100
        
        print("\nDataset information:")
        print(f"Time range: {df.index.min()} to {df.index.max()}")
        print(f"Number of records: {len(df)}")
        
        return df
    
    def prepare_features(self, df, lookback_hours=24):
        """
        Prepare features with hourly lookback periods
        lookback_hours: number of hours to look back for all features
        """
        print("Creating features...")
        result_df = df.copy()
        
        # Initialize feature dictionaries
        time_features = {}
        lagged_features = {}
        rolling_features = {}
        
        # Add time-based features
        time_features.update({
            'hour': df.index.hour,
            'day_of_week': df.index.dayofweek,
            'month': df.index.month,
            'hour_of_day_sin': np.sin(2 * np.pi * df.index.hour / 24),
            'hour_of_day_cos': np.cos(2 * np.pi * df.index.hour / 24),
            'is_weekend': df.index.dayofweek.isin([5, 6]).astype(int)
        })
        
        print("Creating lagged features...")
        # Create lagged features for all historical variables
        for feature_name, feature_data in self.historical_data.items():
            for i in range(1, lookback_hours + 1):
                lagged_features[f'{feature_name}_lag_{i}h'] = feature_data.shift(i)
        
        # Add lagged price features
        for i in range(1, lookback_hours + 1):
            lagged_features[f'price_lag_{i}h'] = df['price'].shift(i)
        
        print("Creating rolling statistics...")
        # Calculate rolling statistics for price and other key metrics
        windows = [24, 168]  # 24 hours and 1 week
        
        for feature_name in ['price'] + list(self.historical_data.keys()):
            data = df['price'] if feature_name == 'price' else self.historical_data[feature_name]
            for window in windows:
                rolling_features.update({
                    f'{feature_name}_rolling_mean_{window}h': data.rolling(window=window).mean().shift(1),
                    f'{feature_name}_rolling_std_{window}h': data.rolling(window=window).std().shift(1)
                })
        
        # Combine all features
        feature_df = pd.DataFrame(
            {**time_features, **lagged_features, **rolling_features},
            index=df.index
        )
        
        # Combine with original price data
        result_df = pd.concat([df[['price']], feature_df], axis=1)
        
        # Handle any infinite values and missing data
        result_df = result_df.replace([np.inf, -np.inf], np.nan)
        result_df = result_df.dropna()
        
        print(f"\nFinal feature set shape: {result_df.shape}")
        print(f"Number of features created: {len(result_df.columns) - 1}")  # -1 for price column
        
        return result_df
    
    def split_data(self, df, test_size=0.2):
        """Split data while preserving time order"""
        train_size = int(len(df) * (1 - test_size))
        train_data = df[:train_size]
        test_data = df[train_size:]
        
        feature_columns = [col for col in df.columns if col != 'price']
        X_train = train_data[feature_columns]
        y_train = train_data['price']
        X_test = test_data[feature_columns]
        y_test = test_data['price']
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test, X_train.columns
    
    def train_models(self, X_train, y_train):
        """Train models with optimized parameters"""
        print("Training Random Forest model...")
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,
            min_samples_split=50,
            min_samples_leaf=20,
            n_jobs=-1,
            random_state=42
        )
        
        print("Training Gradient Boosting model...")
        gb_model = GradientBoostingRegressor(
            n_estimators=100,
            max_depth=8,
            min_samples_split=50,
            min_samples_leaf=20,
            subsample=0.8,
            random_state=42
        )
        
        # Train models
        rf_model.fit(X_train, y_train)
        gb_model.fit(X_train, y_train)
        
        return {'Random Forest': rf_model, 'Gradient Boosting': gb_model}
    
    def evaluate_model(self, model, X_test, y_test, print_results=True):
        """Model evaluation"""
        predictions = model.predict(X_test)
        
        # Calculate metrics
        metrics = {
            'MAE': mean_absolute_error(y_test, predictions),
            'RMSE': np.sqrt(mean_squared_error(y_test, predictions)),
            'MAPE': np.mean(np.abs((y_test - predictions) / y_test)) * 100,
            'R2': r2_score(y_test, predictions)
        }
        
        if print_results:
            print("\nModel Performance Metrics:")
            print(f"MAE: ${metrics['MAE']:.2f}")
            print(f"RMSE: ${metrics['RMSE']:.2f}")
            print(f"MAPE: {metrics['MAPE']:.2f}%")
            print(f"R2 Score: {metrics['R2']:.3f}")
            
            # Calculate prediction intervals
            residuals = y_test - predictions
            residual_std = np.std(residuals)
            print(f"\nPrediction Interval (68% confidence): ±${residual_std:.2f}")
            print(f"Prediction Interval (95% confidence): ±${2*residual_std:.2f}")
        
        return metrics, predictions
    
    def select_important_features(self, X_train, y_train, feature_names, threshold=0.01):
        """Select most important features using Random Forest"""
        print("Selecting important features...")
        
        # Train a random forest for feature selection
        selector = RandomForestRegressor(
            n_estimators=50,
            max_depth=10,
            n_jobs=-1,
            random_state=42
        )
        selector.fit(X_train, y_train)
        
        # Get feature importance
        importance = pd.DataFrame({
            'feature': feature_names,
            'importance': selector.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Select features above threshold
        important_features = importance[importance['importance'] > threshold]
        print(f"\nSelected {len(important_features)} important features out of {len(feature_names)}")
        
        return important_features.feature.tolist()

# Example usage
if __name__ == "__main__":
    predictor = MISOPricePredictor()
    
    # Load and prepare data
    print("Loading data...")
    df = predictor.load_and_prepare_data(
        'combined_generation_hourly_data.csv',
        'miso_lmp_hourly_data_minnesota.csv',
        'temp_data_minnesota.csv'
    )
    
    # Prepare features
    print("Preparing features...")
    df_features = predictor.prepare_features(df)
    
    # Split data (keeping chronological order)
    print("Splitting data...")
    train_size = int(len(df_features) * 0.8)
    
    # Separate features and target
    feature_columns = [col for col in df_features.columns if col != 'price']
    X = df_features[feature_columns]
    y = df_features['price']
    
    # Create train/test sets
    X_train = X[:train_size]
    X_test = X[train_size:]
    y_train = y[:train_size]
    y_test = y[train_size:]
    
    # Select important features
    important_features = predictor.select_important_features(X_train, y_train, feature_columns)
    
    # Use only important features
    X_train = X_train[important_features]
    X_test = X_test[important_features]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train models
    print("\nTraining models...")
    models = predictor.train_models(X_train_scaled, y_train)
    
    # Evaluate models
    for name, model in models.items():
        print(f"\n{name} Evaluation:")
        metrics, predictions = predictor.evaluate_model(model, X_test_scaled, y_test)
        
        # Print feature importance
        if hasattr(model, 'feature_importances_'):
            print("\nTop 10 Most Important Features:")
            importance = pd.DataFrame({
                'feature': important_features,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            print(importance.head(10))

Loading data...
Loading datasets...
Processing timestamps...
Filtering data from 2020-01-01 00:00:00 to 2023-12-31 23:00:00...
Extracting data...

Dataset information:
Time range: 2020-01-01 00:00:00 to 2023-12-31 23:00:00
Number of records: 35064
Preparing features...
Creating features...
Creating lagged features...
Creating rolling statistics...

Final feature set shape: (24941, 595)
Number of features created: 594
Splitting data...
Selecting important features...

Selected 7 important features out of 594

Training models...
Training Random Forest model...
Training Gradient Boosting model...

Random Forest Evaluation:

Model Performance Metrics:
MAE: $9.27
RMSE: $23.08
MAPE: 50.45%
R2 Score: 0.331

Prediction Interval (68% confidence): ±$23.03
Prediction Interval (95% confidence): ±$46.07

Top 10 Most Important Features:
                         feature  importance
0                   price_lag_1h    0.830257
4  temperature_rolling_mean_168h    0.082318
1                    loss_lag_